## A Programmatic Look at the PySAL - ArcGIS Toolbox
### [GitHub Site](https://github.com/Esri/PySAL-ArcGIS-Toolbox)
+ Current functionality focuses on the Spatial Econometrics Methods (spreg module)
+ Includes Spatial Weights Creation/Conversion
+ Leverages the advanced data input-output methodology provided in the Spatial Statistics Toolbox (SSDataObject)
    - See ExtensingArcGISDirectly.ipynb for details on why/how to use the SSDataObject

In [ ]:
import arcpy as ARCPY
import SSDataObject as SSDO
import WeightsUtilities as WU
import pysal as PYSAL
import os as OS

## Standard Data Input-Output based on the Spatial Statistics Data Object
+ Attributes (plus centroids) directly to NumPy Arrays and Pandas Data Frames
+ Quick and easy
+ Note the Unique ID Field (a.k.a MasterField)
 - Used to sort the order in the Arrays with the actual geometries they belong to
 - Cursors are not assured to read in order of attribute table
 - Can use the FID if not using a .swm file
     + *ssdo.oidNAME*

In [ ]:
inputFC = r'../data/CA_Polygons.shp'
fullFC = OS.path.abspath(inputFC)
fullPath, fcName = OS.path.split(fullFC)
ssdo = SSDO.SSDataObject(inputFC)
uniqueIDField = "MYID"
fieldNames = ['GROWTH', 'LOGPCR69', 'PERCNOHS']
ssdo.obtainData(uniqueIDField, fieldNames)
df = ssdo.getDataFrame()
print(df.head())

## Spatial Weights Input-Output
+ Includes methods that create and convert spatial weights matrices to/from PySAL and ArcGIS
    - See NeighborhoodSearching.ipynb for further details

In [ ]:
import Weights as WEIGHTS
import pysal2ArcGIS as PYSAL_UTILS
swmFile = OS.path.join(fullPath, "queen_rs.swm")
kernelSWMFile = OS.path.join(fullPath, "kernel_uniform_4nn.swm")
w = PYSAL_UTILS.swm2Weights(ssdo, swmFile)
kernelW = PYSAL_UTILS.swm2Weights(ssdo, kernelSWMFile)

## Automatic Spatial Econometric Model Search
+ Runs OLS and Diagnostics to determine which spatial econometric model is appropriate

In [ ]:
X = df.as_matrix()
y = X[:,0]
y.shape = ssdo.numObs, 1
autoTestResult = PYSAL_UTILS.autospace(y, X[:,1:], w, kernelW,
                                       opvalue = .1, name_y = fieldNames[0], name_x = fieldNames[1:],
                                       name_w = swmFile, name_gwk = kernelSWMFile, name_ds = ssdo.inName)

In [ ]:
olsModel = autoTestResult['regression1']
print(olsModel.summary)

In [ ]:
finalModel = autoTestResult['regression2']
print(finalModel.summary)

## Simple Output Feature Class Creation Using the Spatial Stats Data Object
+ Declare Candidate Fields
    - Name, Type, Alias, Precision etc...
    - Numpy Array as Data
+ Match Output to correct geometries
+ Copy Input Fields that are not in Python memory using appendFields

In [ ]:
ARCPY.env.overwriteOutput = True
outputFC = r'D:\Data\Projects\PySAL_ArcGIS\PySAL_ArcGIS.gdb\automodel_output'
outFields = {}
outFields['Predy'] = SSDO.CandidateField('Predy', "Double", finalModel.predy)
outFields['Resid'] = SSDO.CandidateField('Resid', "Double", finalModel.u)
appendFields = fieldNames + ["NEW_NAME"]
ssdo.output2NewFC(outputFC, outFields, appendFields = appendFields)